# CAFQA

In [22]:
import sys
sys.path.append("../")
from clapton.clapton import claptonize
from clapton.ansatzes import circular_ansatz
import numpy as np

In [23]:
# define Hamiltonian, e.g. 3q Heisenberg model with random coefficients
paulis = ["XXI", "IXX", "YYI", "IYY", "ZZI", "IZZ"]
np.random.seed(42)  # You can replace 42 with any integer seed
coeffs = np.random.random(len(paulis))
# coeffs = np.ones((len(paulis)))
print(coeffs)

[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452]


In [24]:
# define parametrized Clifford circuit that is being optimized over
# here we use the circular_ansatz template
# we fix 2q gates as they will not be optimized over
vqe_pcirc = circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=True)

In [25]:
# the circuit consists of parametrized gates
for gate in vqe_pcirc.gates:
    print(gate.label, gate.is_fixed())

RY False
RY False
RY False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
RY False
RY False
RY False
RZ False
RZ False
RZ False


In [26]:
# non-fixed gates will be optimized over
# RY and RZ gates can assume 4 values k = 0,1,2,3 which descripe multiples of pi/2

In [27]:
# the initial parameters are all 0
vqe_pcirc.read()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [28]:
# we can look at the corresponding stim circuit
vqe_pcirc.stim_circuit().diagram()

q0: -I-I-X-@-I-I---
         | |
q1: -I-I-|-X-@-I-I-
         |   |
q2: -I-I-@---X-I-I-

In [29]:
# we can assign a different set of parameters
vqe_pcirc.assign([0,1,2,3,0,1,2,3,0,1,2,3])

In [30]:
vqe_pcirc.stim_circuit().diagram()

q0: -I------S_DAG-X-@-Y-S----------------
                  | |
q1: -SQRT_Y-I-----|-X-@-SQRT_Y_DAG-Z-----
                  |   |
q2: -Y------S-----@---X-I----------S_DAG-

In [31]:
# we can perform CAFQA by using the main optimization function "claptonize"
ks_best, _, energy_best = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)

STARTING ROUND 0


started GA at id None with 1 procs

[0, array([-2.66130485, -1.33065243, -1.33065243,  0.        ]), array([0, 2, 2, 2, 0, 2, 3, 3, 3, 3, 3, 3], dtype=object)]
[1, array([-3.41073462, -1.70536731, -1.70536731,  0.        ]), array([0, 1, 2, 0, 3, 1, 2, 2, 2, 2, 3, 0], dtype=object)]
[2, array([-3.41073462, -1.70536731, -1.70536731,  0.        ]), array([0, 1, 2, 0, 3, 1, 2, 2, 2, 2, 3, 0], dtype=object)]
[3, array([-3.41073462, -1.70536731, -1.70536731,  0.        ]), array([1, 3, 0, 0, 2, 0, 3, 0, 2, 2, 2, 2], dtype=object)]
[4, array([-3.41073462, -1.70536731, -1.70536731,  0.        ]), array([1, 3, 0, 0, 2, 0, 3, 0, 2, 2, 2, 2], dtype=object)]
[5, array([-3.41073462, -1.70536731, -1.70536731,  0.        ]), array([3, 1, 2, 1, 2, 0, 3, 3, 3, 0, 0, 0], dtype=object)]
[6, array([-3.41073462, -1.70536731, -1.70536731,  0.        ]), array([3, 1, 2, 1, 2, 0, 3, 3, 3, 0, 0, 0], dtype=object)]
[7, array([-3.41073462, -1.70536731, -1.70536731,  0.        ]), array([0, 1,

In [32]:
# the best parameters are
ks_best

[0, 1, 2, 0, 3, 0, 0, 2, 2, 0, 3, 0]

In [33]:
# with energy
energy_best

np.float64(-1.7053673109431555)

In [34]:
# the corresponding circuit is
vqe_pcirc.assign(ks_best)
vqe_pcirc.stim_circuit().diagram()

q0: -I------I-----X-@-I-I-------
                  | |
q1: -SQRT_Y-S_DAG-|-X-@-Y-S_DAG-
                  |   |
q2: -Y------I-----@---X-Y-I-----

# noisy CAFQA (nCAFQA)

In [35]:
from clapton.depolarization import GateGeneralDepolarizationModel

In [36]:
# let's add a noise model where we specify global 1q and 2q gate errors
nm = GateGeneralDepolarizationModel(p1=0.005, p2=0.02)

In [37]:
vqe_pcirc = circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=True)
vqe_pcirc.add_depolarization_model(nm)

In [38]:
# after every gate a depol channel is added
vqe_pcirc.stim_circuit().diagram()

q0: -I-DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------X-DEPOLARIZE2(0.02)-@-DEPOLARIZE2(0.02)---I-----------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------
                                                                                                                         | |                 | |
q1: ---I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)--------------------|-|-----------------X-DEPOLARIZE2(0.02)-@-DEPOLARIZE2(0.02)-I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)--------------------
                                                                                                                         | |                                     | |
q2: ----------------------I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)-@-DEPOLARIZE2(0.02)---------------------X-DEPOLARIZE2(0.02)--------------------I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)-

In [39]:
# we can perform nCAFQA by using the main optimization function "claptonize"
# now with the noisy circuit
# this is slower, as the noisy circuit needs to be sampled
ks_best, energy_noisy, energy_noiseless = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)

STARTING ROUND 0


started GA at id None with 1 procs

[0, array([-3.20981578, -1.50444847, -1.70536731,  0.        ]), array([1, 3, 0, 2, 2, 2, 0, 1, 3, 2, 2, 2], dtype=object)]
[1, array([-2.49820059, -1.17294616, -1.32525443,  0.        ]), array([3, 2, 1, 0, 0, 1, 1, 1, 0, 0, 0, 3], dtype=object)]
[2, array([-3.21713609, -1.51176878, -1.70536731,  0.        ]), array([0, 3, 0, 0, 2, 3, 0, 1, 3, 2, 3, 3], dtype=object)]
[3, array([-3.23320295, -1.52783564, -1.70536731,  0.        ]), array([1, 3, 0, 2, 2, 2, 0, 1, 3, 2, 2, 2], dtype=object)]
[4, array([-3.23290467, -1.52753736, -1.70536731,  0.        ]), array([3, 1, 0, 3, 0, 0, 0, 1, 3, 2, 2, 2], dtype=object)]
[5, array([-3.23346902, -1.52810171, -1.70536731,  0.        ]), array([1, 3, 0, 2, 2, 2, 0, 1, 3, 2, 2, 2], dtype=object)]
[6, array([-3.23598395, -1.53061664, -1.70536731,  0.        ]), array([1, 3, 0, 2, 2, 2, 0, 1, 3, 1, 2, 2], dtype=object)]
[7, array([-3.23818388, -1.53281657, -1.70536731,  0.        ]), array([0, 3,

In [40]:
# the best parameters are
ks_best

[3, 3, 2, 0, 2, 3, 1, 0, 0, 3, 2, 0]

In [41]:
# with noisy/noiseless energy
energy_noisy, energy_noiseless

(np.float64(-1.509155207904889), np.float64(-1.7053673109431555))

In [42]:
# the corresponding circuit is
vqe_pcirc.assign(ks_best)
vqe_pcirc.snapshot_noiseless().circ_snapshot_noiseless.diagram()

q0: -SQRT_Y_DAG-I-----X-@-SQRT_Y-S_DAG---
                      | |
q1: -SQRT_Y_DAG-Z-----|-X-@------I-----Z-
                      |   |
q2: -Y----------S_DAG-@---X------I-----I-